In [ ]:
import os
import tensorflow as tf
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score

from GlottisNetV2.Utils.DataGeneratorV1 import DataGenerator
from GlottisNetV2.Utils.data import load_data, MAPE_V1, mape_apV1, mape_ppV1
from GlottisNetV2.Utils.Callbacks import get_callbacks
from GlottisNetV2.Models.GlottisNetV1 import glottisnetV1

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Set path to the text file with the image names
coord_train = r"Set path to JSON-file with AP points" #TODO

# Set path to training data
img_training = r" Set path to training images" # TODO

N_train = 200# 55750

# Create video IDs for training data and save them in Pandas Dataframe
cols = ['z','path']
df_imgs_train = pd.DataFrame(columns= cols)
df_segs_train = pd.DataFrame(columns =cols)

for i in tqdm(range(N_train)):
    row_imgs = {'z' : i, 'path': img_training + "\\" + str(i) + ".png"}
    row_segs = {'z': i, 'path': img_training + "\\" + str(i) + '_seg.png'}
    df_imgs_train= df_imgs_train.append(row_imgs, ignore_index = True)
    df_segs_train= df_segs_train.append(row_segs, ignore_index = True)   
print('Created IDs for training images.')

# Save coordinates of anterior and posterior points in Pandas Dataframe
training_data = load_data(coord_train, N_train)
print('Loading of anterior and posterior points finished.')


In [ ]:
'''Training'''

# Set random seed for reproducible training
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
rand=np.random.seed(SEED)
random.seed(SEED)
tf.compat.v1.set_random_seed(SEED)

# Set batch size, learning rate and epochs
BATCH_SIZE = 8 # adjust for your graphics card
LEARNING_RATE = 0.2e-3
EPOCHS = 22
TARGET_HEIGHT = 512
TARGET_WIDTH = 256
SHUFFLE = False
AUGMENT = True
RETURN_MAPS = False
FILTERS = 64
LAYERS = 4
MODEL_PATH = r"Set model path" # TODO
STEPS_PATH = r"Set path to model checkpoints" 'TODO'
N_STEPS = 5 # Save every #N_STEPS epoch

model = glottisnetV1(filters = FILTERS, layers = LAYERS, input_size=(TARGET_HEIGHT, TARGET_WIDTH, 1))

# Hard split of training and validation data 
train_imgs, val_imgs, train_segs, val_segs = train_test_split(df_imgs_train, df_segs_train,  test_size = 0.1, random_state = SEED)

# Datagenerator --> Augmentation and Shuffle
# Training data 
training_generator = DataGenerator(train_imgs, train_segs, batch_size = BATCH_SIZE, target_height = TARGET_HEIGHT, \
                                   target_width = TARGET_WIDTH, shuffle = SHUFFLE, df_coordinates = training_data, \
                                   augment = AUGMENT)

# Validation data
validation_generator = DataGenerator(val_imgs, val_segs, target_height = TARGET_HEIGHT, \
                                     target_width = TARGET_WIDTH, batch_size = BATCH_SIZE, shuffle = False, \
                                     df_coordinates = training_data, augment = False)  


# Compile model with dice_loss for segmentation, mse for prediction maps and use Adam as optimizer
# First exit: coordinates of anterior and posterior points
# Second exit: Segmentations
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = LEARNING_RATE), 
              metrics={'seg': ['acc', iou_score], 'ap_pred': ['acc', MAPE_V1, mape_apV1, mape_ppV1]},
              loss = {'ap_pred': 'mse', 'seg': dice_loss}, run_eagerly = True)


# Train model on dataset and save it
#callbacks = get_callbacks(MODEL_PATH)
model.fit(training_generator, validation_data= validation_generator, epochs = EPOCHS, 
          callbacks = get_callbacks(MODEL_PATH, model, N_STEPS, STEPS_PATH))
